In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
import pandas as pd
import shutil
import os

def fileFiltering():
    df = pd.read_csv(("metadata.csv"))
    findings = df['finding']
    urls = df['filename']
    
    for i in range(len(findings)):
        url = "images/"+urls[i]
        if(findings[i] == "COVID-19"):
            shutil.copy(url,"covid/"+urls[i])
        else:
            shutil.copy(url,"other/"+urls[i])
            
def fileRename():
    covid = os.listdir("covid")
    other = os.listdir("other")
    for i in range(len(covid)):
        os.rename("covid/"+covid[i],"covid_r/covid_"+str(i)+".jpg")
    for i in range(len(other)):
        os.rename("other/"+other[i],"other_r/other_"+str(i)+".jpg")



In [0]:
cd drive/My\ Drive/cronaPoC/

/content/drive/My Drive/cronaPoC


In [0]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf
import os
import random
from skimage import io, transform
import numpy as np
from skimage.color import rgb2gray


In [0]:
batch_size = 5

images = os.listdir("ImageDetetction/xRay")
random.Random(4).shuffle(images)
training_images = images[:500]
test_images = images[500:]

In [0]:
learningRate = 0.001
dropOut = 0.30

height = 200
width = 200

In [0]:
tf.compat.v1.disable_eager_execution()

tf.compat.v1.reset_default_graph()



inputImage = tf.compat.v1.placeholder(tf.float32, shape=[None, height, width,1], name="X")

label = tf.compat.v1.placeholder(tf.float32, shape=[None], name="label")
l_rate = tf.compat.v1.placeholder(tf.float32, name = "learningRate")
d_out = tf.compat.v1.placeholder(tf.float32, name = "dropOut")


regularizer = tf.keras.regularizers.l2()

conv2d_1 = tf.compat.v1.layers.conv2d(inputImage, 
                               filters=8, 
                               kernel_size=[4,4],
                               strides=[1,1], 
                               padding='SAME', 
                               activation="tanh",
                               kernel_regularizer = regularizer,
                               name="conv2d_1")
conv2d_1_dropout = tf.nn.dropout(conv2d_1, rate=d_out)

conv2d_2 = tf.compat.v1.layers.conv2d(conv2d_1_dropout, 
                               filters=10, 
                               kernel_size=[4,4],
                               strides=[1,1], 
                               padding='SAME', 
                               activation="tanh",
                               kernel_regularizer = regularizer,
                               name="conv2d_2")
conv2d_2_dropout = tf.nn.dropout(conv2d_2, rate=d_out)

conv2d_3 = tf.compat.v1.layers.conv2d(conv2d_2_dropout, 
                               filters=10, 
                               kernel_size=[4,4],
                               strides=[1,1], 
                               padding='SAME', 
                               activation="tanh",
                               kernel_regularizer = regularizer,
                               name="conv2d_3")
conv2d_3_dropout = tf.nn.dropout(conv2d_3, rate=d_out)



flat_layer = tf.reshape(conv2d_3, shape=
                        [-1, conv2d_3.shape[1]*conv2d_3.shape[2]*conv2d_3.shape[3]])
fc1 = tf.compat.v1.layers.dense(flat_layer, 64, activation=tf.nn.relu, 
                                kernel_regularizer = regularizer,
                                name="fc1")
fc1_dropout = tf.nn.dropout(fc1, rate=d_out)

fc2 = tf.compat.v1.layers.dense(fc1_dropout, 16, activation=tf.nn.relu, 
                                kernel_regularizer = regularizer,
                                name="fc2")
fc2_dropout = tf.nn.dropout(fc2, rate=d_out)


logit = tf.compat.v1.layers.dense(fc2_dropout, 1, 
                                   kernel_regularizer = regularizer,
                                   name="fc3")
sigmoidal_ouput = tf.math.sigmoid(logit)
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(label, tf.reshape(logit, shape=[-1]))

#Adding Regularization loss
vars   = tf.compat.v1.trainable_variables()
lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in vars
                    if 'bias' not in v.name ]) * 0.001

loss = tf.reduce_mean(tf.math.square(xentropy))
totoal_loss = loss+lossL2

optimizer = tf.compat.v1.train.AdamOptimizer(l_rate)
training_op = optimizer.minimize(totoal_loss)


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.


In [0]:
init = tf.compat.v1.global_variables_initializer()


def binaryConvertor(arr):
    binaryArray = []
    for (index,val) in enumerate(arr):
        if(arr[index]>=0.5):
            binaryArray.append(1)
        else:
            binaryArray.append(0)
    return np.asarray(binaryArray)
   

In [0]:
def get_batch(arr, bsize):
    for batch_i in range(0, 1 + (len(training_images) //bsize)):
        start_i = batch_i * bsize
        try:
            batch = arr[start_i:start_i + bsize]    
        except IndexError:
            batch = arr[start_i:]
        image_list  = [] 
        label = []
        
        for i in range(len(batch)):
            imageFileName = batch[i]
            stringArray = imageFileName.split("_")
            if(stringArray[0] == "covid"):
                label.append(1)
            else:
                label.append(0)
            
            try:
                #print('ImageDetection/xRay/'+imageFileName)
                img = io.imread(imageFileName, plugin='matplotlib')
                img = rgb2gray(img)
                img = transform.resize(img, (200, 200),anti_aliasing=True)#height,width
                #io.imshow(img) 
                #io.show()
                #return
                img = img.reshape(img.shape[0],img.shape[1],1)
                image_list.append(img/255)
                #Normalizing image data
                
            
            except Exception as e:
               print(e)
               pass
        yield np.array(image_list), np.array(label)



In [0]:
cd ../../

/content/drive/My Drive/cronaPoC


In [0]:
cd ImageDetetction/xRay

/content/drive/My Drive/cronaPoC/ImageDetetction/xRay


In [0]:
for index,(data,labels) in enumerate(get_batch(training_images,1)):
  print(data.shape)
  break

print("/ImageDetection/xRay/"+training_images[0])
img = io.imread(training_images[0], plugin='matplotlib')
print(img.shape)

(1, 200, 200, 1)
/ImageDetection/xRay/other_78.jpg
(808, 1232)


In [0]:
n_epochs = 200

trainingLoss = []
testloss = []

with tf.compat.v1.Session() as sess:
    init.run()
    saver = tf.compat.v1.train.Saver()
    for epoch in range(n_epochs):
        i = 0
        total_loss = 0 
        for index,(data,labels) in enumerate(get_batch(training_images,65)):
            if(len(data) == 0):
                continue
            l, prediction,out = sess.run(( totoal_loss,sigmoidal_ouput, training_op), feed_dict={inputImage: data, label: labels, 
                                             l_rate:learningRate,
                                             d_out:dropOut})
            total_loss = total_loss+l
            
        total_loss = total_loss/index
        trainingLoss.append(np.round(total_loss,6))

        print("loss:  ", np.round(total_loss,6))
        binary = binaryConvertor(prediction)
        print("\n\n\nTraining: labels:",np.column_stack((labels,  np.round(prediction,6), binary)))
        oldArray = []
        for index,(data,labels) in enumerate(get_batch(test_images,90)):

            l, out = sess.run((totoal_loss, sigmoidal_ouput), feed_dict={inputImage: data, label: labels, 
                                              l_rate:learningRate,
                                              d_out:0})
            pred = binaryConvertor(out)
            testloss.append(np.round(l,6))
            #oldArray = np.column_stack((oldArray, out))
            print("\n\n\Testing:",np.column_stack((labels, np.round(out,6), pred)))
            break
            # print(val.shape)
           
        #acc_train = accuracy.eval(feed_dict={X: X_bach, y: y_batch})
        #acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        #print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./out/trained")
        

KeyboardInterrupt: ignored

In [0]:
saver = tf.compat.v1.train.Saver()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.compat.v1.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "./out_200/trained")
  print("Model restored.")
  for index,(data,labels) in enumerate(get_batch(test_images,90)):

          l, out = sess.run((totoal_loss, sigmoidal_ouput), feed_dict={inputImage: data, label: labels, 
                                            l_rate:learningRate,
                                            d_out:0})
          pred = binaryConvertor(out)
          testloss.append(np.round(l,6))
          #oldArray = np.column_stack((oldArray, out))
          print("\n\n\Testing:",l, "\n",np.column_stack((labels, np.round(out,6), pred)))
          print(confusion_matrix(labels,pred))
          print(recall_score(labels,pred))
          break

INFO:tensorflow:Restoring parameters from ./out_200/trained
Model restored.


\Testing: 0.20152545 
 [[0.         0.005488   0.        ]
 [0.         0.051046   0.        ]
 [0.         0.104369   0.        ]
 [0.         0.024035   0.        ]
 [0.         0.026679   0.        ]
 [0.         0.140349   0.        ]
 [1.         0.66497898 1.        ]
 [0.         0.025134   0.        ]
 [0.         0.220368   0.        ]
 [0.         0.53733701 1.        ]
 [0.         0.085983   0.        ]
 [0.         0.014057   0.        ]
 [0.         0.031849   0.        ]
 [0.         0.031525   0.        ]
 [1.         0.54931402 1.        ]
 [0.         0.55827898 1.        ]
 [0.         0.347276   0.        ]
 [0.         0.007093   0.        ]
 [0.         0.032473   0.        ]
 [0.         0.003957   0.        ]
 [0.         0.013379   0.        ]
 [0.         0.49167499 0.        ]
 [0.         0.017221   0.        ]
 [1.         0.61585802 1.        ]
 [0.         0.027791   0.        ]

In [0]:
#training vs testing loss
[[0.86302    0.50814497]
 [0.708835   0.52934998]
 [0.696777   0.535662  ]
 [0.68534    0.55519098]
 [0.654384   0.517968  ]
 [0.621354   0.49473199]
 [0.583487   0.432836  ]
 [0.563909   0.42043999]
 [0.54584    0.42336801]
 [0.545241   0.39684999]
 [0.53028    0.43755499]
 [0.52687    0.38712901]
 [0.519641   0.389696  ]
 [0.49697    0.36542699]
 [0.488923   0.39067999]
 [0.466813   0.30290201]
 [0.431677   0.29839399]
 [0.391333   0.257355  ]
 [0.354211   0.21058901]
 [0.314242   0.227961  ]
 [0.325409   0.19589099]
 [0.301792   0.21249899]
 [0.306977   0.226381  ]
 [0.329117   0.23209   ]
 [0.43488    0.27810699]
 [0.514062   0.44012001]
 [0.591335   0.419121  ]
 [0.568405   0.387804  ]
 [0.541846   0.41265801]
 [0.539815   0.36342499]
 [0.453755   0.301256  ]
 [0.37222    0.26464099]
 [0.333743   0.22157501]
 [0.283016   0.19356801]
 [0.245344   0.172203  ]
 [0.235365   0.17085201]
 [0.248675   0.167629  ]
 [0.239941   0.180603  ]
 [0.414558   0.32817799]
 [0.925378   0.37728801]
 [0.730384   0.51049101]
 [0.659182   0.52550697]
 [0.639731   0.451711  ]
 [0.575565   0.36230001]
 [0.50171    0.36106399]
 [0.482589   0.30104101]
 [0.391179   0.266202  ]
 [0.330047   0.23341   ]
 [0.335479   0.24998499]
 [0.318471   0.221908  ]
 [0.279579   0.22177   ]
 [0.271379   0.19483   ]
 [0.263095   0.208473  ]
 [0.262197   0.200627  ]
 [0.273545   0.197713  ]
 [0.317935   0.20725299]
 [0.292217   0.197264  ]
 [0.303727   0.217622  ]
 [0.302747   0.20166101]
 [0.260829   0.189974  ]
 [0.24581    0.16554099]
 [0.221391   0.156991  ]
 [0.206988   0.149673  ]
 [0.198753   0.153754  ]
 [0.197785   0.14672101]
 [0.197635   0.14645401]
 [0.181193   0.14506701]
 [0.191698   0.147036  ]
 [0.182389   0.13423499]
 [0.183353   0.14772201]
 [0.178596   0.142956  ]
 [0.179563   0.152906  ]
 [0.19338    0.154241  ]
 [0.192923   0.158941  ]
 [0.220554   0.16132   ]
 [0.235559   0.183208  ]
 [0.256757   0.17534301]
 [1.13243    0.48183301]
 [1.108156   0.86201602]
 [1.36034    1.34553003]
 [1.779852   1.45423496]
 [1.732874   1.37513804]
 [1.599914   1.24586701]
 [1.422165   1.08418   ]
 [1.244686   0.96763498]
 [1.12859    0.87416798]
 [1.033268   0.83248502]
 [0.942459   0.72539097]
 [0.887455   0.69788003]
 [0.828734   0.64077997]
 [0.788892   0.606713  ]
 [0.75185    0.60663098]
 [0.726736   0.55545902]
 [0.695256   0.51939201]
 [0.679834   0.56076801]
 [0.659502   0.50774699]
 [0.647583   0.49268201]
 [0.611616   0.478894  ]
 [0.606574   0.461016  ]
 [0.579952   0.43850201]
 [0.553006   0.43563399]
 [0.545492   0.369295  ]
 [0.493931   0.34553599]
 [0.475888   0.34524399]
 [0.448047   0.32095701]
 [0.399634   0.27794901]
 [0.387333   0.26125899]
 [0.346524   0.25559199]
 [0.354494   0.245456  ]
 [0.344175   0.247302  ]
 [0.34702    0.219763  ]
 [0.35059    0.248285  ]
 [0.383782   0.263046  ]
 [0.338888   0.22747301]
 [0.330213   0.231869  ]
 [0.313043   0.233522  ]
 [0.306471   0.21642999]
 [0.285052   0.19735201]
 [0.317354   0.222781  ]
 [0.424188   0.253865  ]
 [0.939515   0.48012799]
 [0.736452   0.62414598]
 [0.922924   0.75264603]
 [0.85051    0.613626  ]
 [0.82951    0.59504598]
 [0.672489   0.47224799]
 [0.631583   0.46844801]
 [0.544705   0.41493499]
 [0.468864   0.30928999]
 [0.42863    0.309163  ]
 [0.371872   0.25297701]
 [0.333229   0.24630199]
 [0.301996   0.224452  ]
 [0.280618   0.21084499]
 [0.246366   0.196545  ]
 [0.256082   0.181016  ]
 [0.224673   0.182322  ]
 [0.229484   0.174105  ]
 [0.225171   0.16846   ]
 [0.212012   0.17284299]
 [0.219416   0.16305099]
 [0.207298   0.16943599]
 [0.216059   0.15396699]
 [0.221651   0.177936  ]
 [0.262444   0.16138101]
 [0.294899   0.207118  ]
 [0.329459   0.20777801]
 [0.28751    0.218793  ]
 [0.274387   0.207375  ]
 [0.285804   0.19261999]
 [0.244859   0.18278   ]
 [0.236922   0.176441  ]
 [0.227933   0.16139001]
 [0.191778   0.162374  ]
 [0.22741    0.160157  ]
 [0.206993   0.185084  ]
 [0.213537   0.165316  ]
 [0.209372   0.17348599]
 [0.202622   0.150042  ]
 [0.180083   0.13964   ]
 [0.180545   0.147339  ]
 [0.178139   0.161718  ]
 [0.21044    0.173141  ]
 [0.227822   0.166462  ]
 [0.197175   0.159935  ]
 [0.194639   0.15995   ]
 [0.177725   0.162596  ]
 [0.176959   0.158604  ]
 [0.187062   0.159495  ]
 [0.16843    0.145531  ]
 [0.165475   0.153088  ]
 [0.190575   0.156132  ]
 [0.174694   0.149652  ]
 [0.162689   0.15626401]
 [0.171737   0.148049  ]
 [0.1685     0.147682  ]
 [0.168142   0.14174999]
 [0.165719   0.135344  ]
 [0.1553     0.153276  ]
 [0.185309   0.15304101]
 [0.184806   0.135478  ]
 [0.177363   0.17961   ]
 [0.186485   0.140048  ]
 [0.17918    0.15533499]
 [0.163689   0.149372  ]
 [0.157423   0.153714  ]
 [0.148691   0.148505  ]
 [0.143083   0.14629   ]
 [0.24157    0.198487  ]
 [0.530724   0.242442  ]
 [0.510821   0.334061  ]
 [0.48967    0.34421101]
 [0.434696   0.31972301]
 [0.403667   0.29414901]
 [0.355784   0.25239599]
 [0.315073   0.23468   ]
 [0.29117    0.22749799]
 [0.278177   0.203253  ]
 [0.260423   0.187096  ]
 [0.226243   0.194703  ]]

SyntaxError: ignored